# From normalize notebook

In [1]:
import geopandas as gpd
import time
import zipfile
from getpass import getpass
from glob import glob

import pygbif.occurrences as occ
import pygbif.species as species
import requests

import earthpy
import os
import pathlib
import pandas as pd


In [2]:
%store -r gbif_df 

In [3]:
gbif_gdf = gpd.GeoDataFrame(gbif_df,
    geometry=gpd.points_from_xy(gbif_df.decimalLongitude, gbif_df.decimalLatitude),
    crs="EPSG:4326"
)
gbif_df
print(gbif_gdf.head)

<bound method NDFrame.head of             decimalLatitude  decimalLongitude  month  \
gbifID                                                 
4923516522        41.852812        -87.611721      5   
4444336266        29.755600        -95.739600      4   
4444338185        29.950300        -95.445200      3   
4444335183        29.629800        -95.323900      4   
5196047463        40.003844       -105.028095      5   
...                     ...               ...    ...   
4173223348        44.366800        -83.563700      5   
4444338186        29.806200        -95.557400      4   
4444336188        29.955200        -95.560000     10   
4444336173        30.131200        -95.778500      2   
4444335096        30.146574        -95.642283      4   

                              geometry  
gbifID                                  
4923516522  POINT (-87.61172 41.85281)  
4444336266    POINT (-95.7396 29.7556)  
4444338185    POINT (-95.4452 29.9503)  
4444335183    POINT (-95.3239 29.629

In [4]:
%store -r gbif_gdf
gbif_gdf.columns
gbif_gdf['geometry']


gbifID
4158712344      POINT (-82.5092 41.9137)
4923515059    POINT (-87.61172 41.85281)
4923522410    POINT (-87.61172 41.85281)
4923520798    POINT (-87.61172 41.85281)
4923520314    POINT (-87.63013 41.88036)
                         ...            
4423534780      POINT (-75.1254 40.0626)
4524632357      POINT (-89.8049 44.6967)
4173211734      POINT (-82.4753 42.0478)
4173216429      POINT (-74.5468 46.1638)
4423531794      POINT (-75.1254 40.0626)
Name: geometry, Length: 165614, dtype: geometry

In [7]:
gbif_ecoregion_gdf = (
    gbif_gdf.to_crs(epsg=4326)
    # Match the CRS of the GBIF data and the ecoregions
    
    # Find ecoregion for each observation
    .sjoin(
        gbif_gdf,
        how='inner', 
        predicate='contains')
)
gbif_ecoregion_gdf

,month_left,geometry,gbifID_right,month_right
gbifID_left,,,,
4158712344,5,POINT (-82.5092 41.9137),4160682266,5
4158712344,5,POINT (-82.5092 41.9137),4158712344,5
4158712344,5,POINT (-82.5092 41.9137),4158747133,5
4158712344,5,POINT (-82.5092 41.9137),4160038797,5
4158712344,5,POINT (-82.5092 41.9137),4157001090,5
...,...,...,...,...
4423531794,9,POINT (-75.1254 40.0626),4423529772,9
4423531794,9,POINT (-75.1254 40.0626),4423533803,9
4423531794,9,POINT (-75.1254 40.0626),4423532798,9


In [ ]:
occurrence_df = (
    gbif_ecoregion_gdf
    # Select only necessary columns
    # Reset index
    .reset_index()
    # For each ecoregion, for each month...
    .groupby(['ecoregion_id','month'])
    # ...count the name of occurrences
    .agg
    (occurrences=('gbifID', 'count'),
     area= ('area' ,'first'))
)
 
# Normalize by area
occurrence_df['density'] = (
    occurrence_df.occurrences
    / occurrence_df.area
)
# Get rid of rare observations (possible misidentification?)
occurrence_df = occurrence_df[occurrence_df.occurrences>1]

# Take the mean by ecoregion
mean_occurrences_by_ecoregion = (
    occurrence_df
    .groupby('ecoregion_id')
    .mean()
)
# Take the mean by month
mean_occurrences_by_month = (
    occurrence_df
    .groupby('month')
    .mean()
    
)

In [ ]:
# Normalize by space and time for sampling effort
# Was stuck on this becuase a lack of famliarity 
# Got past it and will remember for net time 
occurrence_df['norm_occurrences'] = (
    occurrence_df[['density']]
    /mean_occurrences_by_ecoregion[['density']]
    /mean_occurrences_by_month[['density']]
)
occurrence_df

In [ ]:
occurrence_df.reset_index().plot.scatter(
    x='month' , y='norm_occurrences' , c='ecoregion_id',
    logy=True
)

# From plot note book

In [ ]:
# Simplify the geometry to speed up processing
eco_gdf.geometry = eco_gdf.simplify(.01, preserve_topology=False)
# Change the CRS to Mercator for mapping
eco_gdf = eco_gdf.to_crs(ccrs.Mercator())

# Check that the plot runs in a reasonable amount of time
eco_gdf.hvplot(geo=True, crs=ccrs.Mercator())

In [ ]:
occurrence_df[['norm_occurrences']].head()

In [ ]:
# Join the occurrences with the plotting GeoDataFrame
occurrence_gdf = eco_gdf.join(occurrence_df[['norm_occurrences']])

# Get the plot bounds so they don't change with the slider
xmin, ymin, xmax, ymax = gbif_gdf.to_crs(ccrs.Mercator()).total_bounds

# Plot occurrence by ecoregion and month
migration_plot = (
    occurrence_gdf
    .to_crs(ccrs.Mercator())
    .hvplot(
        c='norm_occurrences',
        groupby='month',
        # Use background tiles
        geo=True, crs=ccrs.Mercator(), tiles='CartoLight',
        title="Veery Migration",
        xlim=(xmin, xmax), ylim=(ymin, ymax),
        frame_height=600,
        widget_location='bottom'
    )
)

#Save the plot
migration_plot.save('migration.html', embed=True)

# Show the plot
migration_plot

NameError: name 'eco_gdf' is not defined